In [1]:
# install pandas
#!pip install pandas

In [4]:
# count the number of csv files in the financial data folder
import os
import csv
import pandas as pd

# set the path to the PG_financial.xlsx
path = '/Users/senamiruetschlin/Documents/Studies /2. Semester/Guided Studies in Financial Management/Group Project/Investor_Attention/financial_data/stocks/PG_financial.xlsx'

In [5]:
# convert PG_financial.xlsx to a csv file
data_xls = pd.read_excel(path, index_col=None)

# save the data as a csv file
data_xls.to_csv('/Users/senamiruetschlin/Documents/Studies /2. Semester/Guided Studies in Financial Management/Group Project/Investor_Attention/financial_data/stocks/PG_financial.csv', encoding='utf-8')


In [7]:
folder_name = 'google_trends'
file_names = []
for name in file_names:
    file_path = os.path.join(folder_name, f"{name}.csv")
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' does not exist.")
        continue

In [ ]:
updated_list = [   #   'Polkadot price', 
  #  'Polygon price',
   # 'Terra price',
    #    'Stellar price',

    'Tia Token price'
    ]

In [ ]:
google_trends(updated_list)